In [ ]:
import json
import requests
import os
import time

In [ ]:
API_KEY = "ASKMW73735RNXSZCUDNZPFU1W2URIWYXRE"

In [ ]:
import psycopg2
import os
import json

database_dict = {
        "database": os.environ.get("POSTGRES_DB"),
        "user": os.environ.get("POSTGRES_USERNAME"),
        "password": os.environ.get("POSTGRES_PASSWORD"),
        "host": os.environ.get("POSTGRES_WRITER"),
        "port": os.environ.get("POSTGRES_PORT"),
    }
engine = psycopg2.connect(**database_dict)
cur = engine.cursor()

def get_all_members(cur, engine):
    execute_string = f"SELECT * FROM daohaus.member"
    cur.execute(execute_string)
    records = cur.fetchall()
    js = [];
    for x in records:
        curl = []
        curl.append(x[1])
        curl.append(x[6])
        js.append(curl)
    return js

def get_all_daos_chain(cur, engine):
    execute_string = f"SELECT * FROM daohaus.dao"
    cur.execute(execute_string)
    records = cur.fetchall()
    js = {};
    for x in records:
        js[x[0]] = x[5]
    return js

all_members = get_all_members(cur, engine)
all_daos_chains = get_all_daos_chain(cur, engine)

In [19]:
addresses = []
for member in all_members:
    if(all_daos_chains[member[0]] == 1):
        addresses.append(member[1])

In [ ]:
all_tokens = set()
list_of_sets_tokens_in_each_address = []

In [ ]:
counter = 0
for address in addresses:
    cur_add = str(address)
    url = f"https://api.etherscan.io/api?module=account&action=tokentx&address={address}&page=1&offset=100&startblock=0&endblock=27025780&sort=asc&apikey={API_KEY}"
    response = requests.get(url)
    counter+=1
    if(response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json")):
        test_data = response.json()
    else:
        break
        
        
    cd = dict()
    current_address_set_of_tokens = set()
    for transaction in test_data['result']:
        all_tokens.add(transaction['contractAddress'])
        current_address_set_of_tokens.add(transaction['contractAddress'])
        
    cd[str(cur_add)] = current_address_set_of_tokens
    list_of_sets_tokens_in_each_address.append(cd)
    if counter % 20 == 0:
        print("SLEPT")
        time.sleep(1)

In [ ]:
all_tokens_with_spot_prices = {}
for token in all_tokens:
    all_tokens_with_spot_prices[str(token)] = 0
    
did_not_work_tokens = []

In [ ]:
counter = 0
for token in all_tokens:
    fl = True
    response = requests.get(f"https://api.etherscan.io/api?module=token&action=tokeninfo&contractaddress={token}&apikey={API_KEY}")
    if not (response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json")):
        time.sleep(1)
        response = requests.get(f"https://api.etherscan.io/api?module=token&action=tokeninfo&contractaddress={token}&apikey={API_KEY}")
        if not (response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json")):
            time.sleep(1)
            response = requests.get(f"https://api.etherscan.io/api?module=token&action=tokeninfo&contractaddress={token}&apikey={API_KEY}")
            if not (response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json")):
                did_not_work_tokens.append(token)
                fl = False
            else:
                test_data = response.json()
        else:
            test_data = response.json()
    else:
        test_data = response.json()
        
    if fl and test_data['message'] == "OK":
        if(len(test_data['result'])> 0):
            cd = {"tokenPriceUSD": test_data['result'][0]['tokenPriceUSD'], "tokenName" : test_data['result'][0]['tokenName'], "divisor":test_data['result'][0]['divisor'], "symbol" : test_data['result'][0]['symbol']}
            print(test_data['result'][0]['tokenPriceUSD'],test_data['result'][0]['tokenName'],test_data['result'][0]['divisor'])
            all_tokens_with_spot_prices[str(token)] = cd
        else:
            did_not_work_tokens.append(token)
        
    counter+=1
    if(counter % 2 == 1):
        time.sleep(1.5)

In [ ]:
js_str = json.dumps(all_tokens_with_spot_prices)
js_fin = json.loads(js_str)
with open('./../list_of_all_mainnet_tokens_with_spot_prices.json', 'w') as f:
        json.dump(js_fin, f)

In [ ]:
md_of_member_data_with_balances_of_each_token = {}

In [ ]:
counter = 0
for member_dict in list_of_sets_tokens_in_each_address:
    list_of_all_tokens_with_balances_for_current_member = []
    for address in member_dict:
        for token in member_dict[address]:
            if(counter%20 == 0):
                time.sleep(1)
            cd = {}
            response = requests.get(f"https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress={token}&address={address}&tag=latest&apikey={API_KEY}")
            counter += 1
            if not (response.status_code != 204 and response.headers["content-type"].strip().startswith("application/json")):
                response = requests.get(f"https://api.etherscan.io/api?module=account&action=tokenbalance&contractaddress={token}&address={address}&tag=latest&apikey={API_KEY}")
                counter+=1
                test_data = response.json()
            else:
                test_data = response.json()
            cd["contract_address"] = token
            cd["balance"] = test_data['result']
            print(address, token, test_data["result"])
            list_of_all_tokens_with_balances_for_current_member.append(cd)
        
        md_of_member_data_with_balances_of_each_token[address] = list_of_all_tokens_with_balances_for_current_member

In [ ]:
all_wallet_totals = {}
final_data_set = {}

In [ ]:
for address in md_of_member_data_with_balances_of_each_token:
    total = 0
    for token in md_of_member_data_with_balances_of_each_token[address]:
        if(len(md_of_member_data_with_balances_of_each_token[address]) > 0):
            if type(all_tokens_with_spot_prices[token['contract_address']]) is dict:
                price = float(all_tokens_with_spot_prices[token['contract_address']]["tokenPriceUSD"])
                divisor = int(all_tokens_with_spot_prices[token['contract_address']]["divisor"])
                balance = int(token['balance'])
                divided_balance = balance/ (10**divisor)
                value = divided_balance * price
                total+=value
    
    
    all_wallet_totals[address] = total

In [ ]:
ml_of_all_addresses_and_token_info = []

In [ ]:
for address in md_of_member_data_with_balances_of_each_token:
    address_final_dict = {}
    address_final_dict["address"] = address

    enriched_list_of_tokens = []
    for token in md_of_member_data_with_balances_of_each_token[address]:
        if type(all_tokens_with_spot_prices[token['contract_address']]) is dict:
            cd = {}
            cd['contract_address'] = token['contract_address']
            cd['token_name'] = all_tokens_with_spot_prices[token['contract_address']]["tokenName"]
            cd['divisor'] = all_tokens_with_spot_prices[token['contract_address']]["divisor"]
            cd['symbol'] = all_tokens_with_spot_prices[token['contract_address']]["symbol"]
            cd['balance'] = token['balance']
            enriched_list_of_tokens.append(cd)

    address_final_dict["token_info"] = enriched_list_of_tokens
    address_final_dict["wallet_total"] = all_wallet_totals[address]
    ml_of_all_addresses_and_token_info.append(address_final_dict)

In [ ]:
js_str = json.dumps(ml_of_all_addresses_and_token_info)
js_fin = json.loads(js_str)
with open('./all_mainnet_member_with_token_balances_and_wallet_totals.json', 'w') as f:
        json.dump(js_fin, f)